# Git Hooks Examples with Runnable Code

This notebook contains examples of different Git hooks that can be usefull to used to automate tasks in your Git workflow. Each example includes runnable code snippets.

## Table of Contents

1. [Pre-Commit Hook](#1-pre-commit-hook)
2. [Commit-msg Hook](#2-commit-msg-hook)
3. [Post-Commit Hook](#3-post-commit-hook)
4. [Pre-Receive Hook](#4-pre-receive-hook)
5. [Update Hook](#5-update-hook)
6. [Post-Receive Hook](#6-post-receive-hook)

## Installing the hooks

The following describes the manual method. This is, however, necessary only for edits of a pre-commit script file. For other hooks, like the post-commit, we have implemented an automatic installation of the hooks in the install-hooks.sh file. This installation file is called before every commit from the pre-commit hook.

For installation, one needs to copy the hooks files from the 'hooks/' folder to the '.git/hooks/' folder. This can be done by executing the 'install-hooks.sh' Bash script.

Open Git Bash, and type in:

```bash
bash install-hooks.sh
```

To verify the installation has been successful, navigate to the '.git/hooks/' folder and check for our hook files, for example if the post-commit file is present.

The following should also be done automatically by the install-hooks.sh. In case it doesn't do that manually. On some combinations of systems or environments, after the hooks are copied, it might be necessary to run the following command to convert the Windows-style line endings (CRLF) to Unix-style line endings (LF). Otherwise the script won't execute properly, or not at all. The hooks need to be copied to the hooks folder and used before the git commits and converts the line endings itself. The line endings of the scripts need to match the environment that we're commiting from. So, for example, if the hook script was edited on a Windows system, copied to the '.git/hooks/' folder, but not treated with line endings for Linux, and then a Windows Subsystem for Linux (WSL) terminal is used to commit the files, it will throw an error about the hooks. Because the WSL expects LF endings, while the scripts have CRLF endings.

Run the following so the hooks can be executed on Unix-like systems:
```bash
# Run in a (WSL) Bash terminal
dos2unix .git/hooks/post-commit
dos2unix .git/hooks/pre-commit
```

Or, alternatively, the reverse scenario could happen, a Windows user needing to use a script that has LF endings and Git not converting them, yet.
Run the following so the hooks can be executed on Windows-style systems:
```bash
# Run in a Git Bash terminal
unix2dos .git/hooks/post-commit
unix2dos .git/hooks/pre-commit
```
This could be automated in a pre-commit hook, for the other commits. But we're not sure yet, how to solve the automation for the pre-commit hook itself.

## 1. Pre-commit hook

The pre-commit hook is a hook that runs before a commit is made. We can use it to check code quality, run tests, or enforce coding standards.

### Hooks install
We have implemented an automatic install of the hooks. We call the install-hooks.sh from the pre-commit script.

### Code checks
For code quality checks, we show and implement the following pre-commit hook examples. If either of them fails, it aborts the commit, in order for the code quality and tests to be on par before the changes are committed.

- Flake8 - a detector of code style issues and programming errors. Examples of such issues is PEP8 styling violations, or undefined programming names in the code.

- Pytest - looks for any test files in the repo that adhere to naming conventions like test_*.py or *_test.py, and runs them.

- Code coverage - tracks how much of the code is covered by unit tests of Pytest.

Flake8 and Pytest need to be installed in the environment, from which we are making the commits.

```bash
#!/bin/bash
# A Git pre-commit hook example.


#---------------------#
#----- Flag check ----#
#---------------------#

# This check is necessary to not run pre-commit on additional amend commits that happen in the post-commit hook.
# Check if the commit has already been amended by looking for a flag file.
FILE_FLAG=".amend-flag"

if [ -f "$FILE_FLAG" ]; then
    # If the flag file exists, exit to avoid an infinite loop, but don't remove the file, that is done in the post-commit hook
    echo "Exiting the re-triggered pre-commit hook to avoid loop."
    exit 0
fi


#######################
####### Messages ######
#######################

echo "HOOK - PRE-COMMIT - START:"
echo "Your commit is about to happen. Running the pre-commit hooks..."


#######################
#### Hooks install ####
#######################

# Run install-hooks.sh to update Git hooks and adjust line endings
echo "Running install-hooks.sh to update hooks and adjust their line endings..."
bash install-hooks.sh
INSTALL_HOOKS_EXIT_CODE=$?
if [ $INSTALL_HOOKS_EXIT_CODE -ne 0 ]; then
  echo "Failed to run install-hooks.sh. Please check the script and try again."
  echo "Aborting the commit."
  exit 1
fi


#######################
##### Code checks #####
#######################

# Flake8, style guide enforcement
echo "Running Flake8, a detector of code style issues and programming errors..."
flake8 . --exclude=.venv # current directory, excluding the python .venv folder
if [ $? -ne 0 ]; then
  echo "Code issues detected (according to Flake8). Please fix them before committing."
  echo "Aborting the commit."
  exit 1 # stop the commit
fi

# Pytest, unit tests
echo "Running Pytest, unit tests in the repo..."
pytest
PYTEST_EXIT_CODE=$?
if [ $PYTEST_EXIT_CODE -eq 0 ]; then
  echo "Pytest completed successfully."
elif [ $PYTEST_EXIT_CODE -eq 5 ]; then
  echo "No Pytest test was found. Proceeding with the commit."
else
  echo "Pytest unit tests failed with exit code $PYTEST_EXIT_CODE. Please fix the issues before committing."
  echo "Aborting the commit."
  exit 1
fi

# Code coverage check
echo "Checking code coverage of Pytest..."
coverage run -m pytest # the --no-data-file is somehow buggy for us, so omitted, and removing the report manually later
COVERAGE_EXIT_CODE=$?

if [ $COVERAGE_EXIT_CODE -eq 0 ]; then
  echo "Coverage check completed successfully."
elif [ $COVERAGE_EXIT_CODE -eq 5 ]; then
  echo "No data was collected. Proceeding with the commit."
else
  echo "Coverage check failed with exit code $COVERAGE_EXIT_CODE. Please ensure sufficient coverage before committing."
  echo "Aborting the commit."
  exit 1
fi

# Clean up: Remove the .coverage file if it exists
if [ -f ".coverage" ]; then
  rm .coverage
  echo ".coverage file removed after coverage check."
fi


# Display the end message
echo "HOOK - PRE-COMMIT - END"
```

## 2. Commit-msg Hook

The `commit-msg` hook is triggered after the commit message is entered but before the commit is finalized. It can be used to enforce a specific commit message format.

### Example description

**Scenario:** A software development team wants to maintain a consistent format for commit messages, particularly for tracking issues related to project management tools like Jira. They want to ensure that all commit messages start with a specific ticket number format (e.g., `JIRA-123: Description of the change`).

**Pre-Commit Hook Implementation:**

- **Enforce Commit Message Format**: Check that the commit message starts with a JIRA ticket number.

### How to Implement:
1. Navigate to the hooks directory: `cd .git/hooks`
2. Create a new file: `touch commit-msg`
3. Make it executable: `chmod +x commit-msg`
4. Add the below code to the `commit-msg` file.

### Code:


In [ ]:
# This is only example commit-msg hook to enforce message format - addittional changes as testing function need to be made first on specific case
#!/bin/sh

# Get the commit message file path from the first argument
commit_msg_file="$1"

# Check if the commit message file was provided
if [ -z "$commit_msg_file" ]; then
  echo "Error: No commit message file provided."
  echo "Usage: commit-msg <commit-message-file>"
  exit 1
fi

# Check if the commit message starts with "JIRA-123: "
if ! grep -qE "^JIRA-[0-9]+: " "$commit_msg_file"; then
  echo "Error: Commit message must start with a JIRA ticket number, e.g., 'JIRA-123: Fix bug.'"
  exit 1
fi

# If the format is correct, print a success message
echo "Commit message format is correct."

## 4. Pre-Receive Hook

The `pre-receive` hook is a server-side hook that runs before any references are updated. It can be used to enforce policies on incoming pushes.

### Example description

**Scenario:** A team wants to prevent direct pushes to the main branch of their repository to ensure that all changes are reviewed via pull requests. This helps maintain code quality and facilitates collaboration.

**Pre-Receive Hook Implementation:**

- **Block Pushes**: Disallow direct pushes to the main branch.

### How to Implement:
1. Navigate to the hooks directory on the server: `.git/hooks`
2. Create a new file: `touch pre-receive`
3. Make it executable: `chmod +x pre-receive`
4. Add the below code to the `pre-receive` file.

### Code:

In [ ]:
# This is only example pre-receive hook to block pushes to the main branch - addittional changes as testing function need to be made first on specific case
#!/bin/sh

# Read incoming pushes
while read oldrev newrev refname; do
  # Check if the push is to the main branch
  if [ "$refname" = "refs/heads/main" ]; then
    echo "Error: Direct pushes to the main branch are not allowed."
    echo "Please create a pull request instead."
    exit 1
  fi
done

# If no errors, allow the push to proceed
echo "Push accepted."

## 5. Update Hook

The `update` hook runs before a reference is updated. It is useful for enforcing specific policies on branch updates.

### Example description

**Scenario:** A development team wants to prevent force pushes to the main branch to protect the commit history. This ensures that all changes are properly reviewed and merged through pull requests.

**Update Hook Implementation:**

- **Reject Force Pushes**: Disallow force pushes to the `main` branch while allowing normal pushes.

### How to Implement:
1. Navigate to the hooks directory on the server: `.git/hooks`
2. Create a new file: `touch update`
3. Make it executable: `chmod +x update`
4. Add the below code to the `update` file.

### Code:

In [ ]:
# This is only example update hook to reject force pushes to the 'main' branch - addittional changes as testing function need to be made first on specific case
#!/bin/sh

# Read incoming pushes
while read oldrev newrev refname; do
  # Check if the push is to the main branch
  if [ "$refname" = "refs/heads/main" ]; then
    # Check if the old revision is not empty (not a new branch)
    if [ "$oldrev" != "0000000000000000000000000000000000000000" ]; then
      echo "Error: Force pushes to 'main' branch are not allowed."
      exit 1
    fi
  fi
done

# Allow the push to proceed if checks pass
echo "Push accepted."

# Exit with a success status
exit 0

## 6. Post-Receive Hook

The `post-receive` hook runs after a push has been received and processed. It's commonly used for deployment tasks.

### Example description

**Scenario:** A web application is hosted on a server, and the development team wants to automate the deployment process whenever new code is pushed to the repository. This ensures that the latest code is always live without requiring manual intervention.

**Post-Receive Hook Implementation:**

- **Automatic Deployment**: Automatically deploy the latest code to the production environment.

### How to Implement:
1. Navigate to the hooks directory on the server: `.git/hooks`
2. Create a new file: `touch post-receive`
3. Make it executable: `chmod +x post-receive`
4. Add the below code to the `post-receive` file.

### Code:

In [ ]:
# This is only example post-receive hook to deploy to production - addittional changes as testing function need to be made first on specific case
#!/bin/sh

# Deploy the latest code to the specified working tree
if git --work-tree="$WORK_TREE" --git-dir="$GIT_DIR" checkout -f; then
  echo "Deployment successful: Latest code has been deployed to $WORK_TREE."
else
  echo "Error: Deployment failed."
  exit 1
fi